In [1]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig, FlowPolicy, FlowPlacement
from ngraph.lib.algorithms.base import PathAlg, EdgeSelect
from ngraph.failure_manager import FailureManager
from ngraph.failure_policy import FailurePolicy, FailureRule, FailureCondition

In [ ]:
scenario_yaml = """
blueprints:
  server_pod:
    rsw:
        node_count: 48
  
  f16_2tier:
    groups:
      ssw:
        node_count: 36
      fsw:
        node_count: 36

    adjacency:
      - source: /ssw
        target: /fsw
        pattern: mesh
        link_params:
          capacity: 200
          cost: 1
          
  hgrid_2tier:
    groups:
      fauu:
        node_count: 8
      fadu:
        node_count: 36

    adjacency:
      - source: /fauu
        target: /fadu
        pattern: mesh
        link_params:
          capacity: 400
          cost: 1

  fa:
    groups:
      fa1:
        use_blueprint: hgrid_2tier
      fa2:
        use_blueprint: hgrid_2tier
      fa3:
        use_blueprint: hgrid_2tier
      fa4:
        use_blueprint: hgrid_2tier
      fa5:
        use_blueprint: hgrid_2tier
      fa6:
        use_blueprint: hgrid_2tier
      fa7:
        use_blueprint: hgrid_2tier
      fa8:
        use_blueprint: hgrid_2tier
  
  dc_fabric:
    groups:
      plane1:
        use_blueprint: f16_2tier
      plane2:
        use_blueprint: f16_2tier
      plane3:
        use_blueprint: f16_2tier
      plane4:
        use_blueprint: f16_2tier
      plane5:
        use_blueprint: f16_2tier
      plane6:
        use_blueprint: f16_2tier
      plane7:
        use_blueprint: f16_2tier
      plane8:
        use_blueprint: f16_2tier

      pod1:
        use_blueprint: server_pod
      pod36:
        use_blueprint: server_pod
    
    adjacency:
        - source: /pod1/rsw
          target: /plane[0-9]*/fsw/fsw-1
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1
        - source: /pod36/rsw
          target: /plane[0-9]*/fsw/fsw-36
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1
        
network:
  name: "fb_region"
  version: 1.0

  groups:
    dc1:
      use_blueprint: dc_fabric
    dc2:
      use_blueprint: dc_fabric
    dc3:
      use_blueprint: dc_fabric
    dc4:
      use_blueprint: dc_fabric
    dc5:
      use_blueprint: dc_fabric
    dc6:
      use_blueprint: dc_fabric

    fa:
        use_blueprint: fa

  adjacency:
        - source: .*/ssw/.*
          target: .*/fa1/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa2/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa3/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/ssw/.*
          target: .*/fa4/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa5/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa6/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa7/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa8/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1       
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [9]:
len(network.find_links(".*/fadu/.*", ".*/ssw/.*", any_direction=True))

13824

In [10]:
network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/fauu.*",
    mode="combine",
    shortest_path=True,
)

{('.*/fsw.*', '.*/fauu.*'): 2304.0}